<a href="https://colab.research.google.com/github/vkvpgt/checking/blob/master/Bert_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# to connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/1
----------
36
41
14
16
62
65
44
16
16
48
64
46
36
3
21
54
38
63
61
27
50
47
41
37
3
20
27
24
6
63
45
23


TypeError: ignored

In [ ]:
import pickle
output = open('/content/label_encoder.pkl', 'wb')
pickle.dump(label_encoder, output)
output.close()

OSError: ignored

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import pickle 

def BERT(device):

  max_length = 150
  batch_size = 32
  num_classes = 68


  class MultiClassDataset(Dataset):
      def __init__(self, texts, tokenizer, max_length):
          self.texts = texts
          self.tokenizer = tokenizer
          self.max_length = max_length
      
      def __len__(self):
          return len(self.texts)
      
      def __getitem__(self, idx):
          text = str(self.texts[idx])
          inputs = self.tokenizer.encode_plus(
              text,
              add_special_tokens=True,
              max_length=self.max_length,
              padding='max_length',
              truncation=True
          )
          ids = inputs['input_ids']
          mask = inputs['attention_mask']
          return {
              'input_ids': torch.tensor(ids, dtype=torch.long),
              'attention_mask': torch.tensor(mask, dtype=torch.long),
          }


  file_path = '/content/drive/MyDrive/bert_multi_class_classification.pt'
  model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=68)
  model.load_state_dict(torch.load(file_path))
  model.eval()


  data = pd.read_excel("/content/drive/MyDrive/bert_test_with_category.xlsx")
  data = data[:20]
  test_texts = data["text"].tolist()


  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  test_dataset = MultiClassDataset(test_texts, tokenizer, max_length)


  test_loader = DataLoader(test_dataset, batch_size=batch_size)


  predictions = []
  with torch.no_grad():
      for batch in test_loader:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          outputs = model(input_ids, attention_mask)
          print(len(outputs))
          _, batch_predictions = torch.max(outputs.logits, dim=1)
          predictions.extend(batch_predictions.cpu().numpy().tolist())

  file = open('/content/drive/MyDrive/Colab Notebooks/label_encoder.pkl', 'rb')
  label_encoder = pickle.load(file)
  file.close()

  predictions = label_encoder.inverse_transform(predictions)
  return list(predictions)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BERT(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

1


['Interest',
 'Share Sell',
 'Personal Care',
 'Share Sell',
 'Share Sell',
 'Share Sell',
 'Interest',
 'Rent',
 'Share Sell',
 'Share Sell',
 'Share Sell',
 'Personal Care',
 'Purchase by Card',
 'Purchase by Card',
 'Share Sell',
 'Purchase by Card',
 'Personal Care',
 'Share Sell',
 'Share Sell',
 'Personal Care']

In [ ]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import optimization
import os
from tqdm.notebook import tqdm

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
class CreateData(Dataset):

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text, training = True):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.source_text = self.data[source_text]
    self.training = training
    if self.training:
      self.target_text = self.data[target_text]

  def __len__(self):
    return len(self.source_text)

  def __getitem__(self, index):

    source_text = str(self.source_text[index])
    if self.training:
      target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    if self.training:
      target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    
    if self.training:
      target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()

    if self.training:
      target_ids = target['input_ids'].squeeze()
      target_mask = target['attention_mask'].squeeze()

    return_dict={
                'source_ids': source_ids.to(dtype=torch.long), 
                'source_mask': source_mask.to(dtype=torch.long)
               }
    if self.training:
      return_dict['target_ids'] = target_ids.to(dtype=torch.long)
      return_dict['target_ids_y'] = target_ids.to(dtype=torch.long)
    return return_dict

In [ ]:
def T5(tokenizer, model, device, data, model_params):
  """
  Function to evaluate model for predictions

  """
  data.reset_index(inplace=True,drop=True)
  source_text = "text"
  target_text = ""
  test_data = CreateData(data, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text, training=False)

  test_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }

  # Creating dataloader
  loader = DataLoader(test_data, **test_params)


  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(tqdm(loader)):
          #y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          if _%10==0:
              print(f'Completed {_}')

          predictions.extend(preds)
  return predictions

In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00


In [ ]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":16,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":20,              # number of training epochs
    "MAX_SOURCE_TEXT_LENGTH":128,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 
}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = pd.read_excel("/content/drive/MyDrive/bert_test_with_category.xlsx")
data = data[:20]
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/T5/best_model/tokenizer_config.json")
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/T5/best_model")
model = model.to(device)


T5(tokenizer, model, device, data, model_params)


TypeError: ignored